In [1]:
# !uv add pandas scikit-learn

In [2]:
#create a simple tfidf vectorizer and svc classifier
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split, GridSearchCV

In [3]:
data = pd.read_csv("../data/chatbot_classifier_dataset.csv")
oos_data = pd.read_csv("../data/out_of_scope_patterns.csv")
oos_data.rename(columns={"text": "query"}, inplace=True)

data = pd.concat([data, oos_data], ignore_index=True)

# data.rename(columns={"injection_attempts": "injection_patterns",
# 					"inappropriate_content" : "inappropriate_patterns",
# 					}, inplace=True)
data.head()

,query,label
0,I'm looking to order a inexpensive yet high-qu...,shopping_keywords
1,I want to purchase a low-cost Apple laptop.,shopping_keywords
2,What is the cost of the Lenovo gaming console?,shopping_keywords
3,I want to buy a low-cost Samsung monitor.,shopping_keywords
4,"Is the Samsung smartwatch in stock, and how mu...",shopping_keywords


In [4]:
data["label"].value_counts()

label
shopping_keywords        500
policy_keywords          500
product_categories       500
inappropriate_content    500
injection_attempts       500
out_of_scope             500
Name: count, dtype: int64

In [5]:
vectorizer = TfidfVectorizer(max_features=2000,
							ngram_range=(1,2),
							)
vectorizer.fit(data["query"])
X = vectorizer.transform(data["query"])
y = data["label"]

In [6]:
X.shape, y.shape

((3000, 2000), (3000,))

In [7]:
#create simple svm classifier

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

svc = SVC()

In [8]:
# svc_params = {
# 	'C': [0.1, 1, 10],
# 	'kernel': ['linear', 'rbf'],
# 	'gamma': ['scale', 'auto']
# }
# classifier_pipeline = make_pipeline(vectorizer, svc)
svc.fit(X_train, y_train)
svc.score(X_test, y_test)

1.0

In [12]:
svc.predict(vectorizer.transform(["what is the return period for the phone?"]))

array(['policy_keywords'], dtype=object)

In [10]:
#save classifier and vectorizer
import joblib

joblib.dump(svc, "./data/models/guardrail_svc_classifier.joblib")
joblib.dump(vectorizer, "./data/models/guardrail_tfidf_vectorizer.joblib")

['./data/models/guardrail_tfidf_vectorizer.joblib']